# Matrix inversion using Gauss Jordan Reduction Python Sketch

Since it's easier to test and debug Python code, sketch the algorithm first in Python
before converting to a Transform/Feedback program.

https://en.wikipedia.org/wiki/Gaussian_elimination

In [1]:
import feedWebGL2.feedback as fd
import numpy as np
fd.widen_notebook()
np.set_printoptions(precision=2)

In [2]:
# First analogous code in simple Python

def expanded_matrix(square_matrix):
    (n, m) = square_matrix.shape
    assert n == m, "matrix is not square: " + repr(square_matrix.shape)
    result = np.zeros( (n, n+n), dtype=np.float)
    result[:, :n] = square_matrix
    result[:, n:] = np.eye(n)
    return result

N = 5
sq = np.sqrt(np.arange(N*N).reshape((N,N)))-2.4
if 1:
    sq = np.array([
        [0,1,4],
        [1,0,1],
        [0,2,3],
    ])
    N = sq.shape[0]
sq_expanded = expanded_matrix(sq)
sq_expanded

array([[0., 1., 4., 1., 0., 0.],
       [1., 0., 1., 0., 1., 0.],
       [0., 2., 3., 0., 0., 1.]])

In [3]:
def choose_swap_row_and_pivot_value(expanded_matrix, pivot_column_index):
    "Implement in javascript"
    swap_row_index = pivot_column_index
    # Store matrix in column major order to enable column slicing easily from GPU buffer.
    pivot_column = expanded_matrix[:, pivot_column_index]
    pivot_value = 0.0
    nrows = expanded_matrix.shape[0]
    for index in range(pivot_column_index, nrows):
        #value = expanded_matrix[index, pivot_column_index]
        value = pivot_column[index]
        if abs(value) > abs(pivot_value):
            pivot_value = value
            swap_row_index = index
    return (swap_row_index, pivot_value)

choose_swap_row_and_pivot_value(sq_expanded, 1)

(2, 2.0)

In [4]:
choose_swap_row_and_pivot_value(sq_expanded, 2)

(2, 3.0)

In [5]:
def swapped_pivoted_entry(expanded_matrix, irow, icol, ipivot, iswap, pivot_value):
    "Implement as shader."
    # expanded_matrix: texture
    # irow: vertex attribute
    # icol: instance attribute
    # ipivot, iswap, pivot_value: uniforms
    assert pivot_value == expanded_matrix[iswap, ipivot], repr((iswap,ipivot, pivot_value, expanded_matrix[iswap, ipivot]))
    irow0 = irow
    if (icol == ipivot):
        if (irow == ipivot):
            #print (irow, icol, "pivot location", 1.0)
            return 1.0
        else:
            #print (irow, icol, "pivot column", 0.0)
            return 0.0
    if (irow == ipivot):
        value = expanded_matrix[iswap, icol]
        result = value / pivot_value
        #print (irow, icol, "pivot row", result)
        return result
    elif (irow == iswap):
        irow = ipivot
    value = expanded_matrix[irow, icol]
    pivot_column_value = expanded_matrix[iswap, icol]
    factor = pivot_column_value / pivot_value
    pivot_row_value = expanded_matrix[irow, ipivot]
    result = value - factor * pivot_row_value
    #print (irow0, icol, ipivot, iswap, pivot_value, "::", irow, value, pivot_column_value, pivot_row_value, result)
    return result

[swapped_pivoted_entry(sq_expanded, i, 0, 0, 1, 1.0) for i in range(N)]

[1.0, 0.0, 0.0]

In [6]:
def pivot_step(expanded_matrix, ipivot, epsilon=1e-16):
    "Main loop step, implement in javascript."
    result = expanded_matrix.copy()
    (iswap, pivot_value) = choose_swap_row_and_pivot_value(expanded_matrix, ipivot)
    if abs(pivot_value) < epsilon:
        raise ZeroDivisionError("Cannot pivot matrix -- pivot value too small: " + repr((ipivot, iswap, pivot_value)))
    #print(ipivot, "pivotting", iswap, pivot_value)
    (n, m) = result.shape
    for irow in range(n):
        for icol in range(m):
            result[irow, icol] = swapped_pivoted_entry(expanded_matrix, irow, icol, ipivot, iswap, pivot_value)
    return result

print(sq_expanded)
pivot_step(sq_expanded, 0)

[[0. 1. 4. 1. 0. 0.]
 [1. 0. 1. 0. 1. 0.]
 [0. 2. 3. 0. 0. 1.]]


array([[1., 0., 1., 0., 1., 0.],
       [0., 1., 4., 1., 0., 0.],
       [0., 2., 3., 0., 0., 1.]])

In [7]:
def gauss_jordan_invert(square_matrix):
    "Sketch to be implemented using WebGL transform/feedback."
    nrows = square_matrix.shape[0]
    expanded = expanded_matrix(square_matrix)
    for ipivot in range(nrows):
        expanded = pivot_step(expanded, ipivot)
    inverse = expanded[:, nrows:]
    return inverse

A22 = np.array([
    [0, 1],
    [1, 1],
])
A22inv = gauss_jordan_invert(A22)
print (A22inv)
A22inv.dot(A22)

[[-1.  1.]
 [ 1.  0.]]


array([[1., 0.],
       [0., 1.]])

In [8]:
A100 = np.random.random(100).reshape(10, 10)
A100inv = gauss_jordan_invert(A100)
# remove floating point cruft
np.round(10000 * A100.dot(A100inv))/10000

array([[ 1.,  0., -0., -0., -0., -0., -0.,  0., -0., -0.],
       [ 0.,  1.,  0., -0., -0.,  0., -0.,  0., -0.,  0.],
       [ 0.,  0.,  1.,  0.,  0., -0., -0.,  0., -0., -0.],
       [ 0.,  0.,  0.,  1., -0.,  0., -0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1., -0., -0.,  0., -0., -0.],
       [ 0., -0.,  0.,  0.,  0.,  1., -0.,  0., -0.,  0.],
       [ 0.,  0., -0., -0., -0., -0.,  1.,  0., -0., -0.],
       [ 0., -0.,  0.,  0., -0., -0., -0.,  1., -0., -0.],
       [ 0.,  0.,  0., -0., -0., -0., -0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -0., -0.,  0., -0.,  1.]])